In [4]:
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis as PA
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import os
from collections import defaultdict
import pandas as pd
import ipywidgets as widgets
from ipywidgets import *

In [10]:
hydro_dict = {
    #Amino Acids
    'A' : 0.41, #Alanine
    'C' : 0.49, #Cysteine
    'D' : -0.55, #Aspartic acid
    'E' : -0.31, #Glutamic acid
    'F' : 1.00, #Phenylalanine
    'G' : 0.00, #Glycine
    'H' : 0.08, #Histidine
    'I' : 0.99, #Isoleucine
    'K' : -0.23, #Lysine
    'L' : 0.97, #Leucine
    'M' : 0.74, #Methionine
    'N' : -0.28, #Asparagine
    'P' : -0.46, #Proline
    'Q' : -0.10, #Glutamine
    'R' : -0.14, #Arginine
    'S' : -0.05, #Serine
    'T' : 0.13, #Threonine
    'V' : 0.76, #Valine
    'W' : 0.97, #Tryptophan
    'Y' : 0.63, #Tyrosine
    #Variable characters
    'B' : -0.415, #Aspartic acid or Asparagine
    'J' : 0.98, #Leucine or Isoleucine
    'Z' : -0.205 #Glutamic acid or Glutamine
}

def hydroScore(sequencestr):
    score = 0
    for AA in sequencestr:
        score += hydro_dict[AA]
    return(round(score,2))

Values are relative to Glycine (0), values sourced from Monera et al., J. Protein Sci. 1: 319-329 (1995).

In [3]:
def listtostr(strlist):
    output = ""
    for word in strlist:
        output += word + "_"
    return(output[:len(output)-1])
def simplifyname(orgname):
    firstname = [orgname.split("_")[0][0]]
    lastname = orgname.split("_")[1:]
    fullname = firstname + lastname
    return(listtostr(fullname))

In [4]:
datadict = {
    'sequence' : [],
    'ID' : [],
    'description' : [],
    'MW' : [],
    'hydrophobicity' : [],
}

datafile = "data/Yersinia_pestis[632].faa"
with open(datafile,"r") as protfile:
    for record in SeqIO.parse(datafile,"fasta"):
        sequence = str(record.seq)
        parameters = PA(sequence)
        datadict['sequence'].append(sequence)
        datadict['ID'].append(record.id)
        datadict['description'].append(record.description)
        datadict['MW'].append(parameters.molecular_weight())
        datadict['hydrophobicity'].append(hydroScore(sequence))

df = pd.DataFrame.from_dict(datadict)
df.sort_values('hydrophobicity',inplace=True)
df

,sequence,ID,description,MW,hydrophobicity
392,MNRVQFNHHHHHHPD,sp|Q8D079|LPHI_YERPE,sp|Q8D079|LPHI_YERPE His operon leader peptide...,1943.0746,1.17
2835,MWVVLDRVTPGGHGPEDEDNARNNEDNKPSQNGSNFTNSEIDDRIT...,tr|Q7CJ52|Q7CJ52_YERPE,tr|Q7CJ52|Q7CJ52_YERPE Putative uncharacterize...,8985.5158,3.37
1372,MADLYNPDSDREEYELDDKQLYEQVLQLINKLNQDALNK,tr|Q74PL1|Q74PL1_YERPE,tr|Q74PL1|Q74PL1_YERPE Putative colicin immuni...,4686.0812,4.59
3726,MCGWEAPKKAELTGANGGPIQTSNLTPDEAAEAYRKMMG,tr|Q0WJ28|Q0WJ28_YERPE,tr|Q0WJ28|Q0WJ28_YERPE Putative uncharacterize...,4125.6410,5.38
3187,MPIIAPIPRNKRHQMEKIVHKTADKNHSRHLIA,tr|Q9RI14|Q9RI14_YERPE,tr|Q9RI14|Q9RI14_YERPE YPCD1.70c protein (Frag...,3881.5837,5.55
...,...,...,...,...,...
2219,MNKNLYRIVFNQARGMLMVVADIAASGRAASSPSSGVGHTQRRRVS...,tr|Q7CGD9|Q7CGD9_YERPE,tr|Q7CGD9|Q7CGD9_YERPE Putative adhesin OS=Yer...,338786.8022,667.75
1687,MLNYFRAILISWKWKLSHHTSRPHDVKEKGHPRKIKVVAWITLFFQ...,tr|Q7CFY4|Q7CFY4_YERPE,tr|Q7CFY4|Q7CFY4_YERPE Putative invasin OS=Yer...,308572.7179,696.15
1468,MPNGNEMAGFYIDKLSLSQRLSIVSETYDRVNKNNKKEKLKYSYDD...,tr|Q7CGR6|Q7CGR6_YERPE,tr|Q7CGR6|Q7CGR6_YERPE Putative virulence dete...,353076.3328,699.33
1277,MDNLRFSSAPTADSIDASIAQHYPDCEPVAVIGYACHFPESPDGET...,tr|Q9Z373|Q9Z373_YERPE,tr|Q9Z373|Q9Z373_YERPE HMWP1 nonribosomal pept...,348794.3311,765.77


In [5]:
def sequenceCheck(sequence):
    currentAA = 0
    output = []
    while currentAA <= len(sequence)-10:
        segment = sequence[currentAA:currentAA+10]
        output.append(hydroScore(segment))
        currentAA +=1
    return(output)
        
patchData = sequenceCheck('MDSQRNLLLIALLFVSFMIWQAWQVDNNPQPTAQTTQQTTNTATGDKASQAVPGSGQGQLITVKTDVLSLTINTRGGDIEQANLLAYPDTLGSSNTFELLETTPSFVYQAQSGLTGKNGPDNPANGDRPLFEVPQTSFVLADGQDELRIPLTFTSKDGSVFIKTFVLKRNDYAIGVDYHVNNASAAPLELTLFGQLKQSINLPKKRDTGSNNFALQTYRGAAYSSDETKYKKYSFSDIEDKNLDITTKGGWVAMLQQYFATAWIPAANETNTFYSAELGNGLAAIGFKGAPVVIQPGEQKQLSATLWVGPEIQNKMAEIAPHLDLTVDYGWLWFISQPLFKLLKFIHSFVGNWGFSIIVITFIVRGIMYPLTKAQYTSMAKMRLLQPKLAAMRERIGDDKQRMSQEMMALYKAEKVNPLGGCLPLIIQMPIFLALYYMLMSSVELRHAPFILWIHDLSAQDPYYILPILMGITMYFIQKMSPTTVTDPMQQKIMTFMPVIFTVFFLWFPAGLVLYYIVSNLVTILQQQLIYRGLEKRGLHSREKKK')

In [6]:
ct = 0
for num in patchData:
    if num >= 5.00:
        ct += 1
ct

101

In [7]:
datadict = {
    'sequence' : [],
    'ID' : [],
    'description' : [],
    'MW' : [],
    'hydrophobicity' : [],
    'max_patch' : [],
}

datafile = "data/Yersinia_pestis[632].faa"
with open(datafile,"r") as protfile:
    for record in SeqIO.parse(datafile,"fasta"):
        sequence = str(record.seq)
        parameters = PA(sequence)
        datadict['sequence'].append(sequence)
        datadict['ID'].append(record.id)
        datadict['description'].append(record.description)
        datadict['MW'].append(parameters.molecular_weight())
        datadict['hydrophobicity'].append(hydroScore(sequence))
        datadict['max_patch'].append(max(sequenceCheck(sequence)))

df2 = pd.DataFrame.from_dict(datadict)
df2.sort_values('max_patch',inplace=True)
df2

,sequence,ID,description,MW,hydrophobicity,max_patch
392,MNRVQFNHHHHHHPD,sp|Q8D079|LPHI_YERPE,sp|Q8D079|LPHI_YERPE His operon leader peptide...,1943.0746,1.17,1.94
3187,MPIIAPIPRNKRHQMEKIVHKTADKNHSRHLIA,tr|Q9RI14|Q9RI14_YERPE,tr|Q9RI14|Q9RI14_YERPE YPCD1.70c protein (Frag...,3881.5837,5.55,2.32
658,MKRTFQPSVLKRNRSHGFRARMATKNGRQVLARRRAKSRSRLTVSK,sp|Q8Z9U5|RL34_YERPE,sp|Q8Z9U5|RL34_YERPE 50S ribosomal protein L34...,5426.3554,6.62,2.62
3726,MCGWEAPKKAELTGANGGPIQTSNLTPDEAAEAYRKMMG,tr|Q0WJ28|Q0WJ28_YERPE,tr|Q0WJ28|Q0WJ28_YERPE Putative uncharacterize...,4125.6410,5.38,2.66
719,MPVIKVRENEPFDVALRRFKRSCEKAGVLAEVRRREFYEKPTTERK...,sp|P68686|RS21_YERPE,sp|P68686|RS21_YERPE 30S ribosomal protein S21...,8499.8547,9.98,2.95
...,...,...,...,...,...,...
2782,MKILKRLIFICLVIIIIFFLIDCSMQKVAEGRDGYTRSNIWAYYLY...,tr|Q7CJJ4|Q7CJJ4_YERPE,tr|Q7CJJ4|Q7CJJ4_YERPE Putative exported prote...,16269.5674,40.31,9.65
3631,MLSISYEFICSHIKLSWFDIKWAYEKKLIGWHSIVKHAEHLVSLGE...,tr|Q0WE35|Q0WE35_YERPE,tr|Q0WE35|Q0WE35_YERPE Putative uncharacterize...,19843.2810,45.49,9.80
2095,MPNAPKDKVTPQVCSQSTAPLAVGIKTGKKAISHMACLSDRIRAYP...,tr|Q7CG13|Q7CG13_YERPE,tr|Q7CG13|Q7CG13_YERPE Putative membrane prote...,39469.8087,116.04,9.80
1807,MEYWLLIFLLIFLFVIYMQISALSNSVKYIREEIGRLRNELAPRLP...,tr|Q7CIY2|Q7CIY2_YERPE,tr|Q7CIY2|Q7CIY2_YERPE Putative integral membr...,107454.1025,380.82,9.83


In [11]:
def alphaWeighting(sequence):
    currentAA = 7
    output = []
    while currentAA < len(sequence)-7:
        weightedScore = (hydroScore(sequence[currentAA]) + 0.5*(hydroScore(sequence[currentAA+3])+hydroScore(sequence[currentAA-3])+hydroScore(sequence[currentAA+4])+hydroScore(sequence[currentAA-4])+hydroScore(sequence[currentAA+7])+hydroScore(sequence[currentAA-7])))/7
        output.append(weightedScore)
        currentAA +=1
    return(output)

def betaWeighting(sequence):
    currentAA = 4
    output = []
    while currentAA < len(sequence)-4:
        weightedScore = 0.2*(hydroScore(sequence[currentAA]) + hydroScore(sequence[currentAA+2]) + hydroScore(sequence[currentAA-2]) + 0.5*hydroScore(sequence[currentAA+4]) + 0.5*hydroScore(sequence[currentAA-4]))
        output.append(weightedScore)
        currentAA += 1
    return(output)

In [23]:
def countAlpha(sequence):
    scores = alphaWeighting(sequence)
    count = 0
    for n in range(2,len(scores)-2):
        if all([scores[n]>scores[n+2],scores[n]>scores[n-2],scores[n+1]>scores[n+2],scores[n+1]>scores[n-2],scores[n-1]>scores[n+2],scores[n-1]>scores[n-2]]):
            count += 1
    return(count)

def fracAlpha(sequence):
    Np = countAlpha(sequence)
    Nw = len(sequence)
    return(Np / Nw)

In [24]:
fracAlpha('MDSQRNLLLIALLFVSFMIWQAWQVDNNPQPTAQTTQQTTNTATGDKASQAVPGSGQGQLITVKTDVLSLTINTRGGDIEQANLLAYPDTLGSSNTFELLETTPSFVYQAQSGLTGKNGPDNPANGDRPLFEVPQTSFVLADGQDELRIPLTFTSKDGSVFIKTFVLKRNDYAIGVDYHVNNASAAPLELTLFGQLKQSINLPKKRDTGSNNFALQTYRGAAYSSDETKYKKYSFSDIEDKNLDITTKGGWVAMLQQYFATAWIPAANETNTFYSAELGNGLAAIGFKGAPVVIQPGEQKQLSATLWVGPEIQNKMAEIAPHLDLTVDYGWLWFISQPLFKLLKFIHSFVGNWGFSIIVITFIVRGIMYPLTKAQYTSMAKMRLLQPKLAAMRERIGDDKQRMSQEMMALYKAEKVNPLGGCLPLIIQMPIFLALYYMLMSSVELRHAPFILWIHDLSAQDPYYILPILMGITMYFIQKMSPTTVTDPMQQKIMTFMPVIFTVFFLWFPAGLVLYYIVSNLVTILQQQLIYRGLEKRGLHSREKKK')

0.11172161172161173

In [31]:
def countBeta(sequence):
    scores = betaWeighting(sequence)
    count = 0
    for n in range(1,len(scores)-1):
        if bool(scores[n] > scores[n+1]) == bool(scores[n] > scores[n-1]):
            count += 1
    return(count)

def fracBeta(sequence):
    Np = countBeta(sequence)
    Nw = len(sequence)
    return(Np / Nw)

In [34]:
fracBeta('MDSQRNLLLIALLFVSFMIWQAWQVDNNPQPTAQTTQQTTNTATGDKASQAVPGSGQGQLITVKTDVLSLTINTRGGDIEQANLLAYPDTLGSSNTFELLETTPSFVYQAQSGLTGKNGPDNPANGDRPLFEVPQTSFVLADGQDELRIPLTFTSKDGSVFIKTFVLKRNDYAIGVDYHVNNASAAPLELTLFGQLKQSINLPKKRDTGSNNFALQTYRGAAYSSDETKYKKYSFSDIEDKNLDITTKGGWVAMLQQYFATAWIPAANETNTFYSAELGNGLAAIGFKGAPVVIQPGEQKQLSATLWVGPEIQNKMAEIAPHLDLTVDYGWLWFISQPLFKLLKFIHSFVGNWGFSIIVITFIVRGIMYPLTKAQYTSMAKMRLLQPKLAAMRERIGDDKQRMSQEMMALYKAEKVNPLGGCLPLIIQMPIFLALYYMLMSSVELRHAPFILWIHDLSAQDPYYILPILMGITMYFIQKMSPTTVTDPMQQKIMTFMPVIFTVFFLWFPAGLVLYYIVSNLVTILQQQLIYRGLEKRGLHSREKKK')

0.8717948717948718

In [ ]:
def findAlphaMatches(sequence):
    dataDict = {}
    dataDict.keys() = 
    m = n = 0
    while m < len(sequence):
        n = m
        while n < len(sequence):
            segment = sequence[m:n]
            Fp = fracAlpha(segment)
            

In [7]:
params = PA('MDSQRNLLLIALLFVSFMIWQAWQVDNNPQPTAQTTQQTTNTATGDKASQAVPGSGQGQLITVKTDVLSLTINTRGGDIEQANLLAYPDTLGSSNTFELLETTPSFVYQAQSGLTGKNGPDNPANGDRPLFEVPQTSFVLADGQDELRIPLTFTSKDGSVFIKTFVLKRNDYAIGVDYHVNNASAAPLELTLFGQLKQSINLPKKRDTGSNNFALQTYRGAAYSSDETKYKKYSFSDIEDKNLDITTKGGWVAMLQQYFATAWIPAANETNTFYSAELGNGLAAIGFKGAPVVIQPGEQKQLSATLWVGPEIQNKMAEIAPHLDLTVDYGWLWFISQPLFKLLKFIHSFVGNWGFSIIVITFIVRGIMYPLTKAQYTSMAKMRLLQPKLAAMRERIGDDKQRMSQEMMALYKAEKVNPLGGCLPLIIQMPIFLALYYMLMSSVELRHAPFILWIHDLSAQDPYYILPILMGITMYFIQKMSPTTVTDPMQQKIMTFMPVIFTVFFLWFPAGLVLYYIVSNLVTILQQQLIYRGLEKRGLHSREKKK')
structs = params.secondary_structure_fraction

<bound method ProteinAnalysis.secondary_structure_fraction of <Bio.SeqUtils.ProtParam.ProteinAnalysis object at 0x00000291DC821610>>